# Fine-tuning a model with the Trainer API or Keras

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [ ]:
!pip install datasets evaluate transformers[sentencepiece]

<font color = 'lightgreen'>Transformers предоставляет класс Trainer, который поможет вам настроить любую из предварительно обученных моделей, которые он предоставляет для вашего набора данных. После того, как вы выполните всю работу по предварительной обработке данных в последнем разделе, вам останется всего несколько шагов, чтобы определить Trainer. Самая сложная часть, вероятно, будет заключаться в подготовке среды для запуска Trainer.train(), так как он будет работать очень медленно на CPU. Если у вас нет настроенного GPU, вы можете получить доступ к бесплатным GPU или TPU в Google Colab.

Примеры кода ниже предполагают, что вы уже выполнили примеры из предыдущего раздела. Вот краткое резюме того, что вам нужно:

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)


tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Собственно, это то, что было выполнено на предыдущих шагах - данные были токенизированы для процесса оценки парафраз.

## Обучение


<font color = 'lightgreen'>Первый шаг перед тем, как мы сможем определить наш Trainer, — это определить класс TrainingArguments, который будет содержать все гиперпараметры, которые Trainer будет использовать для обучения и оценки. Единственный аргумент, который вам нужно предоставить, — это каталог, в котором будет сохранена обученная модель, а также контрольные точки по пути. Для всего остального вы можете оставить значения по умолчанию, которые должны работать довольно хорошо для базовой тонкой настройки.

In [8]:
from transformers import TrainingArguments

# training_args = TrainingArguments("test-trainer")


training_args = TrainingArguments(
    output_dir="./bert-output",
    report_to="none"  # Отключает wandb
)

## Про wandb

report_to="none"  # Отключает wandb
нужен для того, чтобы не возникало лишних действий с подключением к wandb  
Run data is saved locally in /content/wandb/run-20250220_190013-5qkj1d5z
Syncing run test-trainer to Weights & Biases (docs)
View project at https://wandb.ai/yurkmez-shaikh-individual/huggingface
View run at https://wandb.ai/yurkmez-shaikh-individual/huggingface/runs/5qkj1d5z

При выполненном запуске
Эти сообщения относятся к Weights & Biases (wandb) — инструменту для отслеживания экспериментов с моделями машинного обучения. Давайте разберем их:

1️⃣ "Run data is saved locally in /content/wandb/run-20250220_190013-5qkj1d5z"
Это означает, что все данные текущего запуска (run) сохраняются локально в директории /content/wandb/....
Там хранятся логи, метрики, конфигурация обучения и другие данные.
2️⃣ "Syncing run test-trainer to Weights & Biases (docs)"
wandb автоматически синхронизирует данные этого запуска (test-trainer) с облачным сервисом Weights & Biases.
Это позволяет просматривать графики, логи и метрики в веб-интерфейсе wandb.ai.
3️⃣ "View project at https://wandb.ai/yurkmez-shaikh-individual/huggingface"
Это ссылка на весь проект в Weights & Biases.
Там можно увидеть все запуски (runs), сравнивать их, строить графики и анализировать результаты.
4️⃣ "View run at https://wandb.ai/yurkmez-shaikh-individual/huggingface/runs/5qkj1d5z"
Это ссылка на конкретный запуск (run).
В ней можно увидеть параметры обучения, метрики (loss, accuracy и т. д.), логи, графики и другие данные.

!!!! Если вы не хотите загружать данные в wandb, отключите его:  
training_args = TrainingArguments(
    output_dir="./bert-output",
    report_to="none"  # Отключает wandb
)  


## Далее ...

💡 <font color = 'lightgreen'>Если вы хотите автоматически загружать свою модель в Hub во время обучения, передайте push_to_hub=True в TrainingArguments. Подробнее об этом позже

In [ ]:
training_args

<font color = 'lightgreen'>The second step is to define our model. As in the previous chapter, we will use the `AutoModelForSequenceClassification` class, with two labels:

In [10]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<font color = 'lightgreen'>вы получаете предупреждение после создания экземпляра этой предварительно обученной модели - BERT не был предварительно обучен классификации пар предложений, поэтому заголовок предварительно обученной модели был отброшен, а вместо него был добавлен новый заголовок, подходящий для классификации последовательностей. Предупреждения указывают на то, что некоторые веса не были использованы (те, которые соответствуют удаленному предварительно обученному заголовку), а некоторые другие были инициализированы случайным образом (те, которые предназначены для нового заголовка). В заключение предлагается обучить модель, что мы и собираемся сделать сейчас.  

<font color = 'yellow'>Получив модель, мы можем определить Trainer, передав ему все объекты, созданные до сих пор — модель, training_args, наборы данных для обучения и проверки, наш data_collator и наш tokenizer:

In [11]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    processing_class=tokenizer,
)

<font color = 'lightgreen'>Обратите внимание, что когда вы передаете токенизатор, как мы сделали здесь, data_collator по умолчанию, используемый Trainer, будет DataCollatorWithPadding, как определено ранее, поэтому вы можете пропустить строку data_collator=data_collator в этом вызове. Все равно было важно показать вам эту часть обработки в разделе 2!

<font color = 'yellow'>Чтобы точно настроить модель на нашем наборе данных, нам просто нужно вызвать метод train() нашего Trainer:

In [1]:
import torch
print("GPU доступен:", torch.cuda.is_available())
print("Название GPU:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "Нет GPU")

GPU доступен: True
Название GPU: Tesla T4


In [12]:
trainer.train()

Step,Training Loss
500,0.502800
1000,0.229800


TrainOutput(global_step=1377, training_loss=0.2886179187805131, metrics={'train_runtime': 227.7617, 'train_samples_per_second': 48.314, 'train_steps_per_second': 6.046, 'total_flos': 405114969714960.0, 'train_loss': 0.2886179187805131, 'epoch': 3.0})

<font color = 'lightgreen'>Это запустит тонкую настройку (которая должна занять пару минут на GPU) и сообщит о потере обучения каждые 500 шагов. Однако это не скажет вам, насколько хорошо (или плохо) работает ваша модель. Это потому, что:
Мы не сказали Тренеру оценивать во время обучения, установив Evaluation_strategy на "steps" (оценивать каждые eval_steps) или "epoch" (оценивать в конце каждой эпохи).
Мы не предоставили Тренеру функцию compute_metrics() для расчета метрики во время указанной оценки (иначе оценка просто вывела бы потерю, что не является очень интуитивно понятным числом).

### Evaluation (Оценка)

Давайте посмотрим, как мы можем построить полезную функцию compute_metrics() и использовать ее в следующий раз, когда мы будем тренироваться. Функция должна принимать объект EvalPrediction (который является именованным кортежем с полем predicts и полем label_ids) и будет возвращать словарь, сопоставляющий строки с плавающими числами (строки являются именами возвращаемых метрик, а плавающие числа — их значениями). Чтобы получить некоторые прогнозы из нашей модели, мы можем использовать команду Trainer.predict():

In [13]:
predictions = trainer.predict(tokenized_datasets["validation"])
print(predictions.predictions.shape, predictions.label_ids.shape)

(408, 2) (408,)


Выход метода predict() — это еще один именованный кортеж с тремя полями: predicts, label_ids и metrics. Поле metrics будет содержать только потери в переданном наборе данных, а также некоторые временные метрики (сколько времени потребовалось для прогнозирования, в целом и в среднем). После того, как мы завершим нашу функцию compute_metrics() и передадим ее Trainer, это поле также будет содержать метрики, возвращаемые compute_metrics().

Как вы можете видеть, predicts — это двумерный массив размером 408 x 2 (408 — количество элементов в наборе данных, который мы использовали). Это логиты для каждого элемента набора данных, который мы передали predict() (как вы видели в предыдущей главе, все модели Transformer возвращают логиты). Чтобы преобразовать их в прогнозы, которые мы можем сравнить с нашими метками, нам нужно взять индекс с максимальным значением на второй оси:

In [14]:
import numpy as np

preds = np.argmax(predictions.predictions, axis=-1)

Теперь мы можем сравнить эти pred с метками. Чтобы построить нашу функцию compute_metric(), мы будем полагаться на метрики из библиотеки 🤗 Evaluate. Мы можем загрузить метрики, связанные с набором данных MRPC, так же легко, как мы загрузили набор данных, на этот раз с помощью функции estimate.load(). Возвращенный объект имеет метод compute(), который мы можем использовать для вычисления метрики:

In [15]:
import evaluate

metric = evaluate.load("glue", "mrpc")
metric.compute(predictions=preds, references=predictions.label_ids)

{'accuracy': 0.8553921568627451, 'f1': 0.8998302207130731}

Точные результаты, которые вы получите, могут отличаться, так как случайная инициализация головки модели может изменить достигнутые ею метрики. Здесь мы видим, что наша модель имеет точность 85,78% на проверочном наборе и оценку F1 89,97. Это две метрики, используемые для оценки результатов на наборе данных MRPC для бенчмарка GLUE. Таблица в статье BERT сообщила оценку F1 88,9 для базовой модели. Это была модель без учета, в то время как в настоящее время мы используем модель с учетом, что объясняет лучший результат.

Объединяя все вместе, мы получаем нашу функцию compute_metrics():

In [16]:
def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

И чтобы увидеть, как это используется в действии для отчета о метриках в конце каждой эпохи, вот как мы определяем новый Trainer с помощью этой функции compute_metrics():

Обратите внимание, что мы создаем новый TrainingArguments с его evaluation_strategy, установленным на "epoch", и новую модель — в противном случае мы бы просто продолжили обучение модели, которую мы уже обучили. Чтобы запустить новый тренировочный запуск, мы выполняем:

In [18]:
training_args = TrainingArguments("test-trainer", evaluation_strategy="epoch",  report_to="none")
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-18-61a6c08c965e>:4: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


На этот раз он сообщит о потере проверки и метриках в конце каждой эпохи поверх потери обучения. Опять же, точная оценка точности/F1, которую вы достигнете, может немного отличаться от той, что нашли мы, из-за случайной инициализации модели, но она должна быть в том же диапазоне.

Trainer будет работать из коробки на нескольких GPU или TPU и предоставляет множество опций, таких как обучение со смешанной точностью (используйте fp16 = True в ваших аргументах обучения). Мы рассмотрим все, что он поддерживает, в Главе 10.

На этом завершается введение в тонкую настройку с использованием API Trainer. Пример выполнения этого для большинства распространенных задач NLP будет приведен в Главе 7, но сейчас давайте рассмотрим, как сделать то же самое в чистом PyTorch.

In [19]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.371126,0.857843,0.901024
2,0.527200,0.459970,0.845588,0.893401
3,0.279600,0.774146,0.848039,0.894915


TrainOutput(global_step=1377, training_loss=0.3341553964386319, metrics={'train_runtime': 242.643, 'train_samples_per_second': 45.351, 'train_steps_per_second': 5.675, 'total_flos': 405114969714960.0, 'train_loss': 0.3341553964386319, 'epoch': 3.0})